# Assignment

## Importing Libraries

In [1]:
import numpy as np
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import seaborn as sns

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Currently using {device}')

Currently using cpu


## Loading the Datasets

In [2]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

cifar_dataset_train = datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)

cifar_dataset_test = datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
classes = ()

## Creating the training dataset

In [4]:
from torch.utils.data import DataLoader
batch_size = 100

train_iter = DataLoader(cifar_dataset_train,batch_size,shuffle=True,num_workers=2)

In [5]:
X,y = next(iter(train_iter))

## Creating the test dataset

In [6]:
test_iter = DataLoader(cifar_dataset_test,batch_size,shuffle=False,num_workers=2)

In [7]:
X_test,y_test = next(iter(test_iter))

## Analysing the CIFAR-10 data

In [8]:
print(f'The dimensions of each input image is {X.size()} The number of input channels is {X.size()[1]}')
print(f'The output labels are as follows {y}')

The dimensions of each input image is torch.Size([100, 3, 32, 32]) The number of input channels is 3
The output labels are as follows tensor([3, 3, 0, 9, 4, 9, 8, 2, 5, 4, 3, 5, 6, 4, 7, 2, 1, 6, 3, 9, 9, 2, 1, 5,
        0, 3, 0, 1, 7, 7, 2, 4, 7, 4, 1, 6, 1, 6, 5, 9, 8, 7, 7, 4, 7, 9, 5, 6,
        6, 0, 5, 3, 9, 1, 8, 8, 0, 8, 1, 8, 1, 9, 2, 5, 3, 7, 1, 0, 1, 2, 5, 7,
        5, 6, 3, 1, 7, 5, 1, 5, 8, 6, 3, 2, 0, 7, 6, 3, 7, 3, 3, 1, 0, 6, 6, 9,
        5, 3, 1, 8])


## Example image

## CNN Architecture

In [28]:
class Block(torch.nn.Module):
    def __init__(self,input_channels,output_channels):
        super(Block,self).__init__()

        self.block = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=input_channels,out_channels=output_channels,kernel_size=3,stride=2,padding=1),
            torch.nn.BatchNorm2d(output_channels),
            torch.nn.ReLU(),
        )

        self.shortcut = torch.nn.Sequential(
            torch.nn.Linear(in_features=307200,out_features=16),
            torch.nn.ReLU()
        )

    def forward(self,x):
        shortcut = x

        block = self.block(x)
        shortcut = self.shortcut(torch.flatten(x))
        x = torch.nn.functional.relu(block*shortcut)

        return x

In [35]:
class CNN(torch.nn.Module):
    def __init__(self,num_classes):
        super(CNN,self).__init__()

        self.stem = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3,out_channels=3,kernel_size=3,stride=1,padding=1),
            torch.nn.AvgPool2d(kernel_size=3,stride=1,padding=1),
            torch.nn.ReLU()
        )

        self.block1 = Block(input_channels=3,output_channels=64)
        self.block2 = Block(input_channels=3,output_channels=128)

        self.last = torch.nn.Sequential(torch.nn.AdaptiveAvgPool1d(1000),torch.nn.Flatten())

        self.linear = torch.nn.Sequential(
            torch.nn.Linear(in_features=1000,out_features=num_classes),
            torch.nn.ReLU(),
            torch.nn.Dropout(0),
            torch.nn.Softmax(dim=1)
        )

    def forward(self,x):

        output = self.stem(x)

        o1 = self.block1(output)

        o2 = self.block2(output)

        O = torch.concat((o1,o2),dim=1)
        O = torch.matmul(O,torch.ones(16,1))

        output = self.last(O.view(100,-1))

        output = self.linear(output)

        return output

In [36]:
model = CNN(num_classes=10).to(device)
print(model)

CNN(
  (stem): Sequential(
    (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): AvgPool2d(kernel_size=3, stride=1, padding=1)
    (2): ReLU()
  )
  (block1): Block(
    (block): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (shortcut): Sequential(
      (0): Linear(in_features=307200, out_features=16, bias=True)
      (1): ReLU()
    )
  )
  (block2): Block(
    (block): Sequential(
      (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (shortcut): Sequential(
      (0): Linear(in_features=307200, out_features=16, bias=True)
      (1): ReLU()
    )
  )
  (last): Sequential(
    (0): AdaptiveAvgPool1d(output_size=1000)
    (1): Flatten(start_dim=1, end_dim=-1)


In [37]:
model(X).size()

torch.Size([100, 10])

## Create Loss and Optimization

In [38]:
loss = torch.nn.CrossEntropyLoss()

# Create the optimizer term
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

## Calculate Loss, and Validation Accuracy

In [39]:
def train_test_model(model,train_iter,test_iter,epochs,optimizer,loss):
    for epoch in range(epochs): # Initialize the epochs

        l_score = []

        for i,(X,y) in enumerate(train_iter):

            X = X.to(device)
            y = y.to(device)

            optimizer.zero_grad()

            #model.train()
            y_hat = model(X)
            l = loss(y_hat,y)
            l.backward()
            optimizer.step()

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {l.item():.4f}')

        with torch.no_grad():
            correct = 0
            total = 0
            for data in test_iter:
                X_test,y_test = data
                X_test = X_test.to(device)
                y_test = y_test.to(device)

                y_hat = model(X_test)

                _, predicted = torch.max(y_hat.data,1)
                total += y_test.size(0)
                correct += (predicted == y_test).sum().item()

            print(f'Accuracy of the network on the test images is : {100 * correct // total} percent')

    return

In [40]:
epochs = 30
train_test_model(model,train_iter,test_iter,epochs,optimizer,loss)

Epoch [1/30], Loss: 2.4012
Accuracy of the network on the test images is : 10 percent
Epoch [2/30], Loss: 2.3512
Accuracy of the network on the test images is : 10 percent


KeyboardInterrupt: 